In [1]:
import pandas as pd

df = pd.read_csv('data/NiftySmallCap100Scrips.csv')

exclude = ['KPITTECH','MMTC']

stock_ids = df[~df['Symbol'].isin(exclude) ]['Symbol']


In [2]:
stock_ids

0     AARTIDRUGS
1      AEGISCHEM
2       ALOKINDS
3          AMBER
4         ANURAS
         ...    
95        VIPIND
96     VAKRANGEE
97    WELSPUNIND
98    WOCKPHARMA
99    ZENSARTECH
Name: Symbol, Length: 98, dtype: object

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

AARTIDRUGS
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-1

[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
BEML
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[***********

[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[********************

[*********************100%***********************]  1 of 1 completed
CDSL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[***********

[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
DBL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[************

...

[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
QUESS
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[**********

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
SONATSOFTW
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
TV18BRDCST
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
VAKRANGEE
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[******

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,AARTIDRUGS.NS,2020-08-15,528.85,9.17,545.0,21.94,1947.0,24.10
7,AARTIDRUGS.NS,2020-11-15,716.30,8.08,578.0,26.55,2047.0,26.98
8,AARTIDRUGS.NS,2021-02-15,656.00,7.30,530.0,30.87,2103.0,21.25
9,AARTIDRUGS.NS,2021-05-16,817.50,5.54,502.0,30.09,2155.0,27.17
10,AARTIDRUGS.NS,2021-08-15,649.15,5.27,580.0,26.19,2190.0,24.79
...,...,...,...,...,...,...,...,...
7,ZENSARTECH.NS,2020-11-15,193.60,0.02,979.0,8.10,4009.0,23.90
8,ZENSARTECH.NS,2021-02-15,228.80,6.12,934.0,12.47,3922.0,18.35
9,ZENSARTECH.NS,2021-05-16,268.75,3.91,876.0,13.30,3780.0,20.21
10,ZENSARTECH.NS,2021-08-15,417.90,4.48,937.0,14.53,3726.0,28.76


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-4-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-4-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-4-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,AARTIDRUGS.NS,2020-08-15,528.85,9.17,545.0,21.94,1947.0,24.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AARTIDRUGS.NS,2020-11-15,716.30,8.08,578.0,26.55,2047.0,26.98,9.17,21.94,545.0,24.10,528.85,21.01,14.46,35.47,6.06,-11.89,35.47
8,AARTIDRUGS.NS,2021-02-15,656.00,7.30,530.0,30.87,2103.0,21.25,8.08,26.55,578.0,26.98,716.30,16.27,-24.69,-8.42,-8.30,-9.65,-8.42
9,AARTIDRUGS.NS,2021-05-16,817.50,5.54,502.0,30.09,2155.0,27.17,7.30,30.87,530.0,21.25,656.00,-2.53,27.15,24.62,-5.28,-24.11,24.62
10,AARTIDRUGS.NS,2021-08-15,649.15,5.27,580.0,26.19,2190.0,24.79,5.54,30.09,502.0,27.17,817.50,-12.96,-7.62,-20.58,15.54,-4.87,-20.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ZENSARTECH.NS,2020-11-15,193.60,0.02,979.0,8.10,4009.0,23.90,3.25,11.63,991.0,14.91,173.45,-30.35,41.98,11.63,-1.21,-99.38,11.63
8,ZENSARTECH.NS,2021-02-15,228.80,6.12,934.0,12.47,3922.0,18.35,0.02,8.10,979.0,23.90,193.60,53.95,-35.75,18.20,-4.60,30500.00,18.20
9,ZENSARTECH.NS,2021-05-16,268.75,3.91,876.0,13.30,3780.0,20.21,6.12,12.47,934.0,18.35,228.80,6.66,10.81,17.47,-6.21,-36.11,17.47
10,ZENSARTECH.NS,2021-08-15,417.90,4.48,937.0,14.53,3726.0,28.76,3.91,13.30,876.0,20.21,268.75,9.25,46.23,55.48,6.96,14.58,55.48


In [5]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,BDL.NS,2021-11-15,290.70,-307.02,10.40
7,PRINCEPIPE.NS,2021-11-15,129.91,329.81,30.41
11,BEML.NS,2021-11-15,121.73,-111.34,23.19
11,RITES.NS,2021-11-15,112.68,134.33,9.66
11,COCHINSHIP.NS,2021-11-15,111.55,357.80,3.91
11,PVR.NS,2021-11-15,103.39,-30.30,22.78
2,KALYANKJIL.NS,2021-11-15,96.47,-260.98,NaN
4,ROSSARI.NS,2021-11-15,66.67,6.73,4.30
2,BURGERKING.NS,2021-11-15,63.33,-54.31,NaN
11,EIDPARRY.NS,2021-11-15,60.27,83.58,25.45


In [6]:
df_growth[df_growth['Stock'] == 'ROSSARI.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
0,ROSSARI.NS,2020-11-15,NaN,NaN,NaN
1,ROSSARI.NS,2021-02-15,22.09,4.26,NaN
2,ROSSARI.NS,2021-05-16,3.81,3.85,NaN
3,ROSSARI.NS,2021-08-15,5.96,3.24,NaN
4,ROSSARI.NS,2021-11-15,66.67,6.73,4.3


In [7]:
df_analysis_final.to_csv('data/NiftySC100_FvS_QoQ_Analysis.csv', index=False)

In [8]:
df_growth = pd.read_csv('data/NiftySC100_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,AARTIDRUGS.NS,2020-11-15,716.30,8.08,578.0,26.55,2047.0,26.98,9.17,21.94,545.0,24.10,528.85,21.01,14.46,35.47,6.06,-11.89,35.47
2,AARTIDRUGS.NS,2021-02-15,656.00,7.30,530.0,30.87,2103.0,21.25,8.08,26.55,578.0,26.98,716.30,16.27,-24.69,-8.42,-8.30,-9.65,-8.42
3,AARTIDRUGS.NS,2021-05-16,817.50,5.54,502.0,30.09,2155.0,27.17,7.30,30.87,530.0,21.25,656.00,-2.53,27.15,24.62,-5.28,-24.11,24.62
4,AARTIDRUGS.NS,2021-08-15,649.15,5.27,580.0,26.19,2190.0,24.79,5.54,30.09,502.0,27.17,817.50,-12.96,-7.62,-20.58,15.54,-4.87,-20.58
5,AARTIDRUGS.NS,2021-11-15,552.90,4.60,580.0,22.71,2192.0,24.35,5.27,26.19,580.0,24.79,649.15,-13.29,-1.54,-14.83,0.00,-12.71,-14.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,ZENSARTECH.NS,2020-11-15,193.60,0.02,979.0,8.10,4009.0,23.90,3.25,11.63,991.0,14.91,173.45,-30.35,41.98,11.63,-1.21,-99.38,11.63
560,ZENSARTECH.NS,2021-02-15,228.80,6.12,934.0,12.47,3922.0,18.35,0.02,8.10,979.0,23.90,193.60,53.95,-35.75,18.20,-4.60,30500.00,18.20
561,ZENSARTECH.NS,2021-05-16,268.75,3.91,876.0,13.30,3780.0,20.21,6.12,12.47,934.0,18.35,228.80,6.66,10.81,17.47,-6.21,-36.11,17.47
562,ZENSARTECH.NS,2021-08-15,417.90,4.48,937.0,14.53,3726.0,28.76,3.91,13.30,876.0,20.21,268.75,9.25,46.23,55.48,6.96,14.58,55.48


In [9]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

GNFC.NS          4
FINEORG.NS       4
VAKRANGEE.NS     4
THYROCARE.NS     4
COCHINSHIP.NS    4
BALAMINES.NS     4
LUXIND.NS        4
KEI.NS           4
PRINCEPIPE.NS    4
NBCC.NS          3
CDSL.NS          3
JMFINANCIL.NS    3
RVNL.NS          3
TV18BRDCST.NS    3
CAMS.NS          3
STLTECH.NS       3
KEC.NS           3
RADICO.NS        3
UTIAMC.NS        3
RITES.NS         3
Name: Stock, dtype: int64

In [10]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [11]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
545,VAKRANGEE.NS,2021-11-15,34.42,13.64,-2.73,0,0,0.0,0.0,0.0
214,FINEORG.NS,2021-11-15,22.35,38.94,13.90,0,0,0.0,2.0,2.0
522,TRIDENT.NS,2021-11-15,14.17,12.20,84.85,0,0,0.0,3.0,3.0
516,THYROCARE.NS,2021-11-15,6.67,39.87,-15.52,0,0,2.0,2.0,4.0
312,INTELLECT.NS,2021-11-15,10.78,7.26,8.60,0,0,0.0,4.0,4.0
166,CYIENT.NS,2021-11-15,5.10,5.46,15.23,0,0,4.0,0.0,4.0
291,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
137,CDSL.NS,2021-11-15,24.79,34.53,19.11,0,0,2.0,3.0,5.0
510,TANLA.NS,2021-11-15,34.50,30.34,38.14,0,0,7.0,0.0,7.0
345,KEI.NS,2021-11-15,32.91,36.64,34.93,0,0,4.0,4.0,8.0


In [12]:
df_growth[df_growth['Stock'] == 'THYROCARE.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
512,THYROCARE.NS,2020-11-15,173.21,20275.00,51.16
513,THYROCARE.NS,2021-02-15,-9.80,-24.79,-15.54
514,THYROCARE.NS,2021-05-16,6.52,16.48,15.49
515,THYROCARE.NS,2021-08-15,12.24,47.20,28.61
516,THYROCARE.NS,2021-11-15,6.67,39.87,-15.52
